# Intrinsic Camera Calibration Example

## Import Necessary Packages

In [1]:
import os
import glob
import numpy as np
import cv2 as cv

## Run OpenCV Intrinsic Calibration

This section of code runs the intrinsic calibrations method from the OpenCV library and prints an 'intrinsics' array in a format matching those used by the poseidon_core functions for easy copy and paste.

In [ ]:
# Termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
square_size = 24.5 # mm
objp = np.zeros((9*6,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2) * square_size

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('example_intrinsic_cal_imgs/*.jpg')

# Detection flags for robustness and speed
find_flags = cv.CALIB_CB_ADAPTIVE_THRESH + cv.CALIB_CB_NORMALIZE_IMAGE + cv.CALIB_CB_FAST_CHECK

for i, fname in enumerate(images):
    img = cv.imread(fname)
    if img is None:
        print(f"Warning: Could not read {fname}. Skipping.")
        continue
        
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    
    # Find corners with enhanced flags
    ret, corners = cv.findChessboardCorners(gray, (9,6), find_flags)

    if ret:
        objpoints.append(objp)
        # Refine corner locations to sub-pixel accuracy
        corners2 = cv.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)
        print(f"[{i+1}/{len(images)}] OK: {os.path.basename(fname)}")
    else:
        print(f"[{i+1}/{len(images)}] FAIL: No chessboard found in {os.path.basename(fname)}")

# Verify we have enough data before calibrating
if len(objpoints) > 0:
    # Use the shape of the last successfully processed grayscale image
    h, w = gray.shape[:2]
    ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, (w, h), None, None)
    print(f"\nCalibration successful. Re-projection error: {ret:.4f}")

    # Flatten standard OpenCV output [k1, k2, p1, p2, k3]
    d = dist.ravel()

    # Create the array in custom format
    intrinsics = np.array([
        w,          # number of pixel columns
        h,          # number of pixel rows
        mtx[0,2],   # U component of principal point (cx)
        mtx[1,2],   # V component of principal point (cy)
        mtx[0,0],   # U component of focal length (fx)
        mtx[1,1],   # V component of focal length (fy)
        d[0],       # radial distortion (k1)
        d[1],       # radial distortion (k2)
        d[4],       # radial distortion (k3)
        d[2],       # tangential distortion (p1)
        d[3]        # tangential distortion (p2)
    ])

    # Print intrinsics in easy copy-paste format
    print("\nintrinsics = np.array([")
    labels = ["number of pixel columns", "number of pixel rows", "U component of principal point",
              "V component of principal point", "U component of focal length", "V component of focal length",
              "radial distortion", "radial distortion", "radial distortion",
              "tangential distortion", "tangential distortion"]
    for val, label in zip(intrinsics, labels):
        # Format to 4 decimal places for floats, integers for standard ints
        val_str = f"{int(val)}" if label.startswith("number of") else f"{val:.4f}"
        print(f"    {val_str},  # {label}")
    print("])")

else:
    print("\nError: No usable calibration images found.")

In [ ]:
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
    mean_error += error
 
print( "Total Error (pixels): {}".format(mean_error/len(objpoints)) )